# Basic RAG System with LangChain




### Load Dependencies

In [ ]:
!pip install langchain==0.3.10
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11
!pip install jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.2.2
    Uninstalling langsmith-0.2.2:
      Successfully uninstalled langsmith-0.2.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.11
    Uninstalling langchain-0.3.11:
      Successfully uninstalled langchain-0.3.11
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.

In [ ]:
!pip install langchain-chroma==0.1.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 6.7 MB/s eta 0:00:

## Enter API Tokens

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Enter your OpenAI Key: ')

Enter your OpenAI Key: ··········


## Setup Environment Variables

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

### Load Wikipedia Data

In [ ]:
!gdown 1aZxZejfteVuofISodUrY2CDoyuPLYDGZ

Downloading...
From: https://drive.google.com/uc?id=1aZxZejfteVuofISodUrY2CDoyuPLYDGZ
To: /content/rag_docs.zip
100% 5.92M/5.92M [00:00<00:00, 45.9MB/s]


In [ ]:
!unzip rag_docs.zip

Archive:  rag_docs.zip
   creating: rag_docs/
  inflating: rag_docs/attention_paper.pdf  
  inflating: rag_docs/cnn_paper.pdf  
  inflating: rag_docs/resnet_paper.pdf  
  inflating: rag_docs/vision_transformer.pdf  
  inflating: rag_docs/wikidata_rag_demo.jsonl  


In [ ]:
from langchain.document_loaders import JSONLoader

loader = JSONLoader(file_path='./rag_docs/wikidata_rag_demo.jsonl',
                    jq_schema='.',
                    text_content=False,
                    json_lines=True)
passages = loader.load()

In [ ]:
len(passages)

1801

In [ ]:
passages[3]

Document(metadata={'source': '/content/rag_docs/wikidata_rag_demo.jsonl', 'seq_num': 4}, page_content='{"id": "71548", "title": "Chi-square distribution", "paragraphs": ["In probability theory and statistics, the chi-square distribution (also chi-squared or formula_1\\u00a0 distribution) is one of the most widely used theoretical probability distributions. Chi-square distribution with formula_2 degrees of freedom is written as formula_3. It is a special case of gamma distribution.", "Chi-square distribution is primarily used in statistical significance tests and confidence intervals. It is useful, because it is relatively easy to show that certain probability distributions come close to it, under certain conditions. One of these conditions is that the null hypothesis must be true. Another one is that the different random variables (or observations) must be independent of each other."]}')

In [ ]:
type(passages[3])

langchain_core.documents.base.Document

### Load Open AI LLMs

In [ ]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

### Generate LLM Embeddings and store them in Chroma Vector DB

**Chroma Vector DB** is a versatile, open-source vector database designed for managing and querying vector embeddings. It is easy to set up and integrates well with various AI tools and algorithms. Chroma is particularly useful for applications that require rapid and precise retrieval of content represented as embeddings—efficient data formats for text, images, and soon, audio and video.

**Key Features:**
- **Integration with AI Tools:** Chroma supports embedding functions from leading providers like OpenAI, Google, and Hugging Face, allowing for flexible and powerful data handling.
- **Ease of Use:** The database provides default embedding functions, or users can integrate external APIs to generate embeddings.
- **Efficient Querying:** Users can create collections to store embeddings, documents, and metadata. These can be queried to retrieve the most similar items, making information retrieval quick and effective.
- **Flexible API:** Chroma offers a straightforward API that supports both standard operations and custom embedding functions.

For more detailed information, visit the official Chroma documentation [here](https://docs.trychroma.com).


In [ ]:
passages[:3]

[Document(metadata={'source': '/content/rag_docs/wikidata_rag_demo.jsonl', 'seq_num': 1}, page_content='{"id": "84801", "title": "Chinese New Year", "paragraphs": ["Chinese New Year, known in China as the SpringFestival and in Singapore as the LunarNewYear, is a holiday on and around the new moon on the first day of the year in the traditional Chinese calendar. This calendar is based on the changes in the moon and is only sometimes changed to fit the seasons of the year based on how the Earth moves around the sun. Because of this, Chinese New Year is never on January1. It moves around between January21 and February20.", "The Chinese New Year is of the most important holidays for Chinese people all over the world. Its 7th day used to be used instead of birthdays to count people\'s ages in China. The holiday is still used to tell people which \\"animal\\" of the Chinese zodiac they are part of. The holiday is a time for gifts to children and for family gatherings with large meals, just l

{"id": "87272", "title": "Thelema", "paragraphs": ["Thelema is the English spelling of the Greek noun : \\"will\\", from the verb \\"\\u03b8\\u03ad\\u03bb\\u03c9\\": to will, wish, purpose. Early Christian writings use the word to refer to the will of God, the human will, and even the will of God\'s opponent, the Devil.", "Thelema is also a way of life first written about by Fran\\u00e7ois Rabelais (16th century) in his famous books of fiction, Gargantua and Pantagruel. The core of this way of life was summed up in the phrase \\"\\"Do what thou wilt\\"\\" (\\"fay \\u00e7e que vouldras\\" in the original old French). This idea was later put into practice in the mid 18th century by Sir Francis Dashwood at Medmenham.", "This Thelemic Law of Rabelais was revived by Aleister Crowley in 1904 when Crowley wrote \\"The Book of the Law\\". This book contains both the word \\"Thelema\\" in Greek as well as the phrase \\"Do what thou wilt.\\" From this, Crowley took Thelema as the name of his own religion. Thus Shri Gurudev Mahendranath wrote that Rabelais, Dashwood, and Crowley must share the honor of perpetuating Thelema."]}

In [ ]:
import json
from langchain.docstore.document import Document
from tqdm import tqdm

processed_docs = []

for doc in tqdm(passages):
    processed_doc = json.loads(doc.page_content)
    metadata = {
        "title" : processed_do      c['title'],
        "id" : processed_doc['id']
    }
    document_text = ' '.join(processed_doc['paragraphs'])
    processed_docs.append(Document(page_content=document_text,
                                   metadata=metadata))


100%|██████████| 1801/1801 [00:00<00:00, 34934.91it/s]


In [ ]:
processed_docs[:3]

[Document(metadata={'title': 'Chinese New Year', 'id': '84801'}, page_content='Chinese New Year, known in China as the SpringFestival and in Singapore as the LunarNewYear, is a holiday on and around the new moon on the first day of the year in the traditional Chinese calendar. This calendar is based on the changes in the moon and is only sometimes changed to fit the seasons of the year based on how the Earth moves around the sun. Because of this, Chinese New Year is never on January1. It moves around between January21 and February20. The Chinese New Year is of the most important holidays for Chinese people all over the world. Its 7th day used to be used instead of birthdays to count people\'s ages in China. The holiday is still used to tell people which "animal" of the Chinese zodiac they are part of. The holiday is a time for gifts to children and for family gatherings with large meals, just like Christmas in Europe and in other Christian areas. Unlike Christmas, the children usually 

In [ ]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [ ]:
# The vectorstore we'll be using
from langchain_chroma import Chroma

# The splitting and chunking strategy
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=3000,
                                          chunk_overlap=200)
chunked_docs = splitter.split_documents(processed_docs)

In [ ]:
chunked_docs[:3]

[Document(metadata={'title': 'Chinese New Year', 'id': '84801'}, page_content='Chinese New Year, known in China as the SpringFestival and in Singapore as the LunarNewYear, is a holiday on and around the new moon on the first day of the year in the traditional Chinese calendar. This calendar is based on the changes in the moon and is only sometimes changed to fit the seasons of the year based on how the Earth moves around the sun. Because of this, Chinese New Year is never on January1. It moves around between January21 and February20. The Chinese New Year is of the most important holidays for Chinese people all over the world. Its 7th day used to be used instead of birthdays to count people\'s ages in China. The holiday is still used to tell people which "animal" of the Chinese zodiac they are part of. The holiday is a time for gifts to children and for family gatherings with large meals, just like Christmas in Europe and in other Christian areas. Unlike Christmas, the children usually 

## Create Vector DB and Retriever

If you have already created `wiki_db`in the previous hands-on session then just load the DB and DO NOT run the following code to create the database again, ignore this when running on Colab

In [ ]:
# create vector DB of docs and embeddings - takes 1 min on Colab
chroma_db = Chroma.from_documents(documents=chunked_docs,
                                  collection_name='wiki_db',
                                  embedding=openai_embed_model,
                                  # need to set the distance function to cosine else it uses euclidean by default
                                  # check https://docs.trychroma.com/guides#changing-the-distance-function
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./wiki_db")

## Load Vector DB from disk

Run the following code if your vector DB already exists on disk from the previous hands-on session

In [ ]:
# load from disk
chroma_db = Chroma(persist_directory="./wiki_db",
                   collection_name='wiki_db',
                   embedding_function=openai_embed_model)

In [ ]:
chroma_db

In [ ]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                              search_kwargs={"k": 5, "score_threshold": 0.2})

In [ ]:
similarity_retriever.invoke('what is machine learning?')

[Document(metadata={'id': '564928', 'title': 'Machine learning'}, page_content='Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science. The idea came from work in artificial intelligence. Machine learning explores the study and construction of algorithms which can learn and make predictions on data. Such algorithms follow programmed instructions, but can also make predictions or decisions based on data. They build a model from sample inputs. Machine learning is done where designing and programming explicit algorithms cannot be done. Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.'),
 Document(metadata={'id': '359370', 'title': 'Supervised learning'}, page_content='In machine learning, supervised learning is the task of inferring a function from labelled

In [ ]:
similarity_retriever.invoke('what is ai?')

[Document(metadata={'id': '6360', 'title': 'Artificial intelligence'}, page_content='Artificial intelligence (AI) is the ability of a computer program or a machine to think and learn. It is also a field of study which tries to make computers "smart". They work on their own without being encoded with commands. John McCarthy came up with the name "Artificial Intelligence" in 1955. In general use, the term "artificial intelligence" means a programme which mimics human cognition. At least some of the things we associate with other minds, such as learning and problem solving can be done by computers, though not in the same way as we do. Andreas Kaplan and Michael Haenlein define AI as a system’s ability to correctly interpret external data, to learn from such data, and to use those learnings to achieve specific goals and tasks through flexible adaptation. An ideal (perfect) intelligent machine is a flexible agent which perceives its environment and takes actions to maximize its chance of su

In [ ]:
similarity_retriever.invoke('who won the champions league?')

[Document(metadata={'id': '639943', 'title': '2017–18 UEFA Europa League'}, page_content="The 2017–18 UEFA Europa League was the 47th season of Europe's secondary club football tournament organised by UEFA, and the 9th season since it was renamed from the UEFA Cup to the UEFA Europa League. The last game was played at the Parc Olympique Lyonnais in Décines-Charpieu, France. Atlético Madrid won against Marseille to win their third Europa League title. As winners, Atlético Madrid played against the winners of the 2017–18 UEFA Champions League, Real Madrid, in the 2018 UEFA Super Cup. They also automatically qualified for the 2018–19 UEFA Champions League group stage, but since they had already qualified through their league performance, the berth reserved was given to the third-placed team of the 2017–18 Ligue 1."),
 Document(metadata={'id': '3070', 'title': 'Chelsea F.C.'}, page_content="Chelsea F.C. is an English football club that plays in the English Premier League. Their home stadiu

### Build a QA RAG Chain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If the answer is not present in the context, just say that you don't know.
            Keep the answer to the point.

            Question:
            {question}

            Context:
            {context}

            Answer:
         """

prompt_template = ChatPromptTemplate.from_template(prompt)

In [ ]:
prompt_template.pretty_print()

================================ Human Message =================================

You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If the answer is not present in the context, just say that you don't know.
            Keep the answer to the point.

            Question:
            {question}

            Context:
            {context}

            Answer:
         


## RAG Chain - Using LCEL

In [ ]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (similarity_retriever
                      |
                    format_docs),
        "question": RunnablePassthrough()
    }
       |
    prompt_template
      |
    chatgpt
)

In [ ]:
from IPython.display import display, Markdown
query = "What is machine learning?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

Machine learning is a subfield of computer science that gives computers the ability to learn from data without being explicitly programmed. It involves the study and construction of algorithms that can learn and make predictions or decisions based on data, building models from sample inputs.

In [ ]:
query = "Who was the winner of the champions league in 2020?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

I don't know.

In [ ]:
query = "What is AI?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

Artificial intelligence (AI) is the ability of a computer program or a machine to think and learn, and it is a field of study aimed at making computers "smart." AI systems can interpret external data, learn from it, and use those learnings to achieve specific goals through flexible adaptation.

In [ ]:
query = "How is it different from statistics"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

I don't know.

# Conversational RAG System with LangChain

In many Q&A applications, the ability to engage in back-and-forth conversations with users is crucial. This necessitates the application having a form of "memory" to recall past interactions and apply this context to current queries.

This guide focuses on integrating historical messages into the application's logic. Additional details on managing chat history can be found [here](https://python.langchain.com/docs/expression_language/how_to/message_history/).

![](https://i.imgur.com/8hLJMPl.gif)

### Building on the Q&A RAG System - to a Conversational Q&A RAG System

We will enhance our Q&A RAG System, which utilizes the Wikipedia dataset, by implementing the following updates:

- **Prompt Adjustment:** Our prompt will be modified to include historical messages as inputs, allowing the system to maintain context over the course of a conversation.

- **Contextualizing Questions:** We will introduce a sub-chain mechanism to reformulate the latest user query by considering the chat history. This is crucial for understanding questions that refer back to previous messages. For example, a query like "Can you elaborate on the second point?" relies on the context provided by preceding interactions, which affects the system's ability to retrieve relevant information effectively.





## Contextualizing the Question

To maintain a seamless flow in conversations, especially in a Q&A setting, it's essential to incorporate historical interactions. Here’s how we achieve this:

### Defining a Sub-Chain for Historical Context

1. **Sub-Chain Creation:** We'll define a sub-chain that uses both historical messages and the latest user query. This sub-chain reformulates the question if it refers to any past interactions, ensuring the system, especially the vector database understands the context to return the most relevant documents to this newly reworded question.

2. **Using `MessagesPlaceholder`:** Our prompt construction involves a `MessagesPlaceholder` variable named `chat_history`. This setup allows us to input a list of messages using the `chat_history` key. The system integrates these messages, positioning them after its own responses and before the latest user question.

3. **Helper Function Usage:** We employ the `create_history_aware_retriever` function available [here](https://api.python.langchain.com/en/latest/chains/langchain.chains.history_aware_retriever.create_history_aware_retriever.html). This function is crucial for handling instances where the chat history might be empty and orchestrates the sequence of operations: `prompt | llm | StrOutputParser() | retriever`.

4. **Chain Construction:** The `create_history_aware_retriever` constructs a chain that processes inputs under the keys `input` and `chat_history`, ensuring the output schema aligns with that of a retriever.

By implementing these steps, our system can effectively utilize historical context to better understand and respond to user queries, thereby enhancing the conversational experience.


In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

rephrase_system_prompt = """Given a chat history and the latest user question
which might reference context in the chat history, formulate a standalone question
which can be understood without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""

rephrase_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rephrase_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    chatgpt, similarity_retriever, rephrase_prompt
)

In [ ]:
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7a914af45ed0>, search_type='similarity_score_threshold', search_kwargs={'k': 5, 'score_threshold': 0.2}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotat

This chain prepends a rephrasing of the input query to our retriever, so that the retrieval incorporates the context of the conversation.

## Building the QA RAG Chain with Chat History

Now we're ready to construct our comprehensive QA RAG chain, which leverages historical context for more accurate and relevant responses.

### Components of the QA RAG Chain

1. **Creating Document Chains:**
   - We use the `create_stuff_documents_chain` function, which is detailed [here](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html). This function is used to create a `question_answer_chain`, accepting inputs such as `context`, `chat_history`, and `input`. It efficiently combines the retrieved context with the conversation history and the current query to generate an informed answer.

2. **Building the Final QA RAG Chain:**
   - The entire QA RAG chain is assembled using the `create_retrieval_chain` function, available [here](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval.create_retrieval_chain.html). This chain integrates the `history_aware_retriever` with the `question_answer_chain`. It retains intermediate outputs like the retrieved context for added convenience during the query handling process.
   - The `create_retrieval_chain` function accepts keys such as `input` and `chat_history` and includes `input`, `chat_history`, `context`, and `answer` in its outputs.

By implementing these steps, the system not only contextualizes but also provides accurate answers by synthesizing information from both the current and historical interactions. This method enhances the conversational AI’s ability to understand and respond to user queries dynamically, making the interactions more engaging and relevant.


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks.
                      Use the following pieces of retrieved context to answer the question.
                      Also refer to the whole historical conversation if you need additional
                      context to answer the question
                      If the answer is not present in the context and historical conversation,
                      just say that you don't know.
                      Keep the answer to the point.

                      Context:
                      {context}

                  """

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", """Question:
                     {input}

                     Answer:
                  """),
    ]
)

question_answer_chain = create_stuff_documents_chain(chatgpt, qa_prompt)
question_answer_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[l

In [ ]:
qa_rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
qa_rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7a914af45ed0>, search_type='similarity_score_threshold', search_kwargs={'k': 5, 'score_threshold': 0.2}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langch

In [ ]:
from IPython.display import display, Markdown
chat_history = []

question = "What is machine learning?"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
display(Markdown(response['answer']))

Machine learning is a subfield of computer science that gives computers the ability to learn without being explicitly programmed. It involves the study and construction of algorithms that can learn from data and make predictions or decisions based on that data. Machine learning is used in situations where designing and programming explicit algorithms is not feasible, with applications including spam filtering, network intrusion detection, optical character recognition, search engines, and computer vision.

In [ ]:
chat_history

[]

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history.extend([HumanMessage(content=question),
                     AIMessage(content=response["answer"])])
chat_history

[HumanMessage(content='What is machine learning?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Machine learning is a subfield of computer science that gives computers the ability to learn without being explicitly programmed. It involves the study and construction of algorithms that can learn from data and make predictions or decisions based on that data. Machine learning is used in situations where designing and programming explicit algorithms is not feasible, with applications including spam filtering, network intrusion detection, optical character recognition, search engines, and computer vision.', additional_kwargs={}, response_metadata={})]

In [ ]:
question = "How is it different from statistics"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
display(Markdown(response['answer']))

Machine learning and statistics differ primarily in their focus and approach. Statistics is concerned with data collection, organization, analysis, interpretation, and presentation, often emphasizing inference and making predictions based on sample data. It relies on established mathematical theories and models to understand data relationships.

In contrast, machine learning focuses on building algorithms that can learn from data and improve their performance over time without being explicitly programmed. While both fields use data, machine learning often deals with larger datasets and aims to create models that can generalize well to new, unseen data. Additionally, machine learning emphasizes prediction and decision-making, whereas statistics often emphasizes understanding and interpreting data relationships.

In [ ]:
chat_history.extend([HumanMessage(content=question),
                     AIMessage(content=response["answer"])])
chat_history

[HumanMessage(content='What is machine learning?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Machine learning is a subfield of computer science that gives computers the ability to learn without being explicitly programmed. It involves the study and construction of algorithms that can learn from data and make predictions or decisions based on that data. Machine learning is used in situations where designing and programming explicit algorithms is not feasible, with applications including spam filtering, network intrusion detection, optical character recognition, search engines, and computer vision.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How is it different from statistics', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Machine learning and statistics differ primarily in their focus and approach. Statistics is concerned with data collection, organization, analysis, interpretation, and presentation, often emphasizing 

In [ ]:
question = "What is the chi square test?"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
display(Markdown(response['answer']))

The chi-square test is a statistical test used to determine whether there is a significant association between categorical variables. It compares the observed frequencies in each category of a contingency table to the frequencies expected under the null hypothesis of no association. The test calculates a chi-square statistic, which measures how much the observed counts deviate from the expected counts. A high chi-square value indicates a significant difference between observed and expected frequencies, leading to the rejection of the null hypothesis. The chi-square test is commonly used in tests of independence and goodness-of-fit.

In [ ]:
response

{'input': 'What is the chi square test?',
 'chat_history': [HumanMessage(content='What is machine learning?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Machine learning is a subfield of computer science that gives computers the ability to learn without being explicitly programmed. It involves the study and construction of algorithms that can learn from data and make predictions or decisions based on that data. Machine learning is used in situations where designing and programming explicit algorithms is not feasible, with applications including spam filtering, network intrusion detection, optical character recognition, search engines, and computer vision.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How is it different from statistics', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Machine learning and statistics differ primarily in their focus and approach. Statistics is concerned with data collection, organization, 

In [ ]:
chat_history.extend([HumanMessage(content=question),
                     AIMessage(content=response["answer"])])

In [ ]:
chat_history

[HumanMessage(content='What is machine learning?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Machine learning is a subfield of computer science that gives computers the ability to learn without being explicitly programmed. It involves the study and construction of algorithms that can learn from data and make predictions or decisions based on that data. Machine learning is used in situations where designing and programming explicit algorithms is not feasible, with applications including spam filtering, network intrusion detection, optical character recognition, search engines, and computer vision.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How is it different from statistics', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Machine learning and statistics differ primarily in their focus and approach. Statistics is concerned with data collection, organization, analysis, interpretation, and presentation, often emphasizing 

In [ ]:
chat_history[-4:]

[HumanMessage(content='How is it different from statistics', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Machine learning and statistics differ primarily in their focus and approach. Statistics is concerned with data collection, organization, analysis, interpretation, and presentation, often emphasizing inference and making predictions based on sample data. It relies on established mathematical theories and models to understand data relationships.\n\nIn contrast, machine learning focuses on building algorithms that can learn from data and improve their performance over time without being explicitly programmed. While both fields use data, machine learning often deals with larger datasets and aims to create models that can generalize well to new, unseen data. Additionally, machine learning emphasizes prediction and decision-making, whereas statistics often emphasizes understanding and interpreting data relationships.', additional_kwargs={}, response_metadata={}),
 Hu

In [ ]:
question = "How is it different from the t-test"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history[-2:]})
chat_history.extend([HumanMessage(content=question),
                     AIMessage(content=response["answer"])])
display(Markdown(response['answer']))

The chi-square test and the t-test are different in several ways:

1. **Type of Data**: The chi-square test is used for categorical data, while the t-test is used for continuous data.

2. **Purpose**: The chi-square test assesses the association between categorical variables or tests the goodness-of-fit of observed data to a theoretical distribution. The t-test compares the means of two groups to determine if they are significantly different from each other.

3. **Test Statistic**: The chi-square test uses the chi-square statistic, which is based on the difference between observed and expected frequencies. The t-test uses the t statistic, which is based on the difference between sample means relative to the variability of the samples.

4. **Assumptions**: The chi-square test assumes that the observations are independent and that the expected frequency in each category is sufficiently large. The t-test assumes that the data is normally distributed and that the variances of the two group

In [ ]:
response

{'input': 'How is it different from the t-test',
 'chat_history': [HumanMessage(content='What is machine learning?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Machine learning is a subfield of computer science that gives computers the ability to learn without being explicitly programmed. It involves the study and construction of algorithms that can learn from data and make predictions or decisions based on that data. Machine learning is used in situations where designing and programming explicit algorithms is not feasible, with applications including spam filtering, network intrusion detection, optical character recognition, search engines, and computer vision.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How is it different from statistics', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Machine learning and statistics differ primarily in their focus and approach. Statistics is concerned with data collection, organiz

# Multi-User Conversational RAG System with LangChain

In many Q&A applications, the ability to engage in back-and-forth conversations with users is crucial. This necessitates the application having a form of "memory" to recall past interactions and apply this context to current queries.

However in most real-world conversational systems, multiple users or user sessions will be accessing the system simultaneously.

![](https://i.imgur.com/X4WivLu.gif)

Here we will show how you can use `SQLChatMessageHistory` such that we can store separate conversation histories per user or session which is often the need for real-world chatbots which will be accessed by many users at the same time. Instead of in-memory we can store it in a SQL database which can be used to store a lot of conversations.

We use a `get_session_history` function which is expected to take in a `session_id` and return a Message History object. Everything is stored in a SQL database. This `session_id` is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain

We also use a `memory_buffer_window` function to only use the top-K last historical conversations before sending it to the LLM, basically our own implementation of `ConversationBufferWindowMemory`




In [ ]:
# removes the memory database file - usually not needed
# you can run this only when you want to remove all conversation histories
!rm memory.db

rm: cannot remove 'memory.db': No such file or directory


In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

######### REPHRASER ############
rephrase_system_prompt = """Given a chat history and the latest user question
which might reference context in the chat history, formulate a standalone question
which can be understood without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is."""

rephrase_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rephrase_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    chatgpt, similarity_retriever, rephrase_prompt
)



######### MULTI_USER RAG RESPONSE GENERATOR ############
qa_system_prompt = """You are an assistant for question-answering tasks.
                      Use the following pieces of retrieved context to answer the question.
                      Also refer to the whole historical conversation if you need additional
                      context to answer the question
                      If the answer is not present in the context and historical conversation,
                      just say that you don't know.
                      Keep the answer to the point.

                      Context:
                      {context}
                  """

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", """Question:
                     {input}

                     Answer:
                  """),
    ]
)

# used to retrieve conversation history from database
# based on a specific user or session ID
def get_session_history_db(session_id):
    history = SQLChatMessageHistory(session_id, "sqlite:///memory.db")
    return history

# subset historical conversations based on last K conversation messages
# here by default we use the last 10 conversations (ai-human) as memory to the input prompt
def memory_buffer_window(messages, lastk_conversations=10):
    return messages[-(lastk_conversations*2):] # each conversation has 2 messages - (human prompt, AI response)

# custom RAG chain which looks at last K conversational messages
question_answer_chain = (
    RunnablePassthrough.assign(chat_history=lambda x: memory_buffer_window(x["chat_history"]))
      |
    qa_prompt
      |
    chatgpt
      |
    StrOutputParser()
)
qa_rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


############ CONVERSATIONAL RAG CHAIN ####################
conversational_rag_chain = RunnableWithMessageHistory(
    qa_rag_chain,
    get_session_history_db,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
from IPython.display import display, Markdown

def conv_rag_chatbot(usersession_id, prompt):
    response = conversational_rag_chain.invoke(
                                {"input": prompt},
                                config={
                                    "configurable": {"session_id": usersession_id}
                                }
    )
    print('Answer:')
    display(Markdown(response['answer']))
    print('Sources:')
    for document in response['context']:
        print(document)
        print()

    return response

In [ ]:
us_id = 'bond007'
r = conv_rag_chatbot(us_id, 'Explain what is machine learning')

/usr/local/lib/python3.10/dist-packages/langchain_core/runnables/history.py:608: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(


Answer:


Machine learning is a subfield of computer science that gives computers the ability to learn without being explicitly programmed. It involves the study and construction of algorithms that can learn from data and make predictions or decisions based on that data. Machine learning is particularly useful in situations where designing and programming explicit algorithms is not feasible. Examples of its applications include spam filtering, network intrusion detection, optical character recognition, search engines, and computer vision.

Sources:
page_content='Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science. The idea came from work in artificial intelligence. Machine learning explores the study and construction of algorithms which can learn and make predictions on data. Such algorithms follow programmed instructions, but can also make predictions or decisions based on data. They build a model from sample inputs. Machine learning is done where designing and programming explicit algorithms cannot be done. Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.' metadata={'id': '564928', 'title': 'Machine learning'}

page_content='In machine learning, supervised learning is the task of inferring a function from labelled training data. The results of the training are known beforehand, the sy

In [ ]:
r = conv_rag_chatbot(us_id, 'Now explain what is AI')

Answer:


Artificial intelligence (AI) is the ability of a computer program or a machine to think and learn. It is a field of study that aims to make computers "smart," enabling them to work independently without being explicitly programmed with commands. AI systems can interpret external data, learn from it, and use those learnings to achieve specific goals and tasks through flexible adaptation. In general, AI refers to programs that mimic human cognition, such as learning and problem-solving, although they do so in ways that differ from human thought processes.

Sources:
page_content='Artificial intelligence (AI) is the ability of a computer program or a machine to think and learn. It is also a field of study which tries to make computers "smart". They work on their own without being encoded with commands. John McCarthy came up with the name "Artificial Intelligence" in 1955. In general use, the term "artificial intelligence" means a programme which mimics human cognition. At least some of the things we associate with other minds, such as learning and problem solving can be done by computers, though not in the same way as we do. Andreas Kaplan and Michael Haenlein define AI as a system’s ability to correctly interpret external data, to learn from such data, and to use those learnings to achieve specific goals and tasks through flexible adaptation. An ideal (perfect) intelligent machine is a flexible agent which perceives its environment and takes actions to maximize its chance of success at some goal or objective. As machines become increasing

In [ ]:
us_id = 'jim003'
r = conv_rag_chatbot(us_id, 'What is linear regression?')

Answer:


Linear regression is a statistical method used to model the relationship between a dependent variable and one or more independent variables by fitting a linear equation to observed data. The goal is to find the best-fitting straight line (linear predictor function) that can predict the value of the dependent variable based on the values of the independent variables. In linear regression, the coefficients of the independent variables are termed regression coefficients, and the model aims to minimize the difference between the observed values and the values predicted by the linear equation.

Sources:
page_content='In statistics and in machine learning, a linear predictor function is a linear function (linear combination) of a set of coefficients and explanatory variables (independent variables), whose value is used to predict the outcome of a dependent variable. Functions of this sort are standard in linear regression, where the coefficients are termed regression coefficients. However, they also occur in various types of linear classifiers (e.g. logistic regression, perceptrons, support vector machines, and linear discriminant analysis), as well as in various other models, such as principal component analysis and factor analysis. In many of these models, the coefficients are referred to as "weights".' metadata={'id': '430252', 'title': 'Linear predictor function'}

page_content='Logistic Regression, also known as "Logit Regression" or "Logit Model", is a mathematical model used in statistics to estimate (guess) the probability of an event occurring having been given some p

In [ ]:
us_id = 'bond007'
r = conv_rag_chatbot(us_id, 'what about deep learning and how is it different from the previous two')

Answer:


Deep learning is a subset of machine learning that primarily uses neural networks with multiple layers (known as deep neural networks) to process data. It is particularly effective for tasks such as speech recognition, image recognition, and handwriting recognition, which are challenging for traditional algorithms. 

The key differences are:
1. **Structure**: Deep learning models have multiple hidden layers that allow them to learn increasingly abstract representations of data, while traditional machine learning models typically have fewer layers.
2. **Data Requirements**: Deep learning often requires large amounts of data to perform well, whereas traditional machine learning can work with smaller datasets.
3. **Complexity**: Deep learning models are generally more complex and computationally intensive than traditional machine learning models.

In summary, while all three—AI, machine learning, and deep learning—are interconnected, deep learning is a more specialized approach within machine learning that leverages complex neural networks to handle large-scale data and perform sophisticated tasks.

Sources:
page_content='Deep learning (also called deep structured learning or hierarchical learning) is a kind of machine learning, which is mostly used with certain kinds of neural networks. As with other kinds of machine-learning, learning sessions can be unsupervised, semi-supervised, or supervised. In many cases, structures are organised so that there is at least one intermediate layer (or hidden layer), between the input layer and the output layer. Certain tasks, such as as recognizing and understanding speech, images or handwriting, is easy to do for humans. However, for a computer, these tasks are very difficult to do. In a multi-layer neural network (having more than two layers), the information processed will become more abstract with each added layer. Deep learning models are inspired by information processing and communication patterns in biological nervous systems; they are different from the structural and functional properties of biological brains (especially the human br

In [ ]:
us_id = 'jim003'
r = conv_rag_chatbot(us_id, 'how is it different from logistic regression')

Answer:


Linear regression predicts continuous outcomes and assumes a linear relationship between the dependent and independent variables, resulting in a straight line equation. In contrast, logistic regression is used for binary outcomes (0 or 1) and models the probability of an event occurring using the logistic function, which produces an S-shaped curve. Logistic regression does not assume a linear relationship; instead, it estimates the probability of the dependent variable being 1 based on the independent variables.

Sources:
page_content='Logistic Regression, also known as "Logit Regression" or "Logit Model", is a mathematical model used in statistics to estimate (guess) the probability of an event occurring having been given some previous data. Logistic Regression works with binary data, where either the event happens (1) or the event does not happen (0). So given some feature x it tries to find out whether some event y happens or not. So y can either be 0 or 1. In the case where the event happens, y is given the value 1. If the event does not happen, then y is given the value of 0. For example, if y represents whether a sports team wins a match, then y will be 1 if they win the match or y will be 0 if they do not. This is known as "Binomial Logistic Regression". There is also another form of Logistic Regression which uses multiple values for the variable y. This form of Logistic Regression is known as "Multinomial Logistic Regression". Logistic Regression uses the logistic function to find a mod